# Notebook 3: 快速开始

此 notebook 介绍了`bigdl-llm`的基本用法，并带您逐步构建一个最基础的聊天应用程序。

## 3.1 安装 `bigdl-llm`

如果您尚未安装bigdl-llm，请按照以下示例进行安装。这一行命令将安装最新版本的`bigdl-llm`以及所有常见LLM应用程序开发所需的依赖项。

In [ ]:
!pip install --pre --upgrade bigdl-llm[all]

## 3.2 加载预训练模型

在使用LLM之前，您首先需要加载一个模型。这里我们以一个相对较小的LLM作为示例，即[open_llama_3b_v2](https://huggingface.co/openlm-research/open_llama_3b_v2)。

## 3.3 优化模型

通常情况下，您只需要一行`optimize_model`就可以轻松优化已加载的任何PyTorch模型。

模型加载过程如下：

首先，使用您喜欢的任何PyTorch API来加载您的模型。这里，我们使用`Hugging Face Transformers`库的`LlamaForCausalLM`来加载`open_llama_3b_v2`。

然后，调用`optimize_model`来优化已加载的模型（默认采用INT4优化）。

In [3]:
from transformers import LlamaForCausalLM
from bigdl.llm import optimize_model

model_path = 'openlm-research/open_llama_3b_v2'

model = LlamaForCausalLM.from_pretrained(model_path,
                                         torch_dtype="auto",
                                         low_cpu_mem_usage=True)

# With only one line to enable BigDL-LLM optimization on model
model = optimize_model(model)

> **注意**
>
> * 如果您希望使用除了INT4以外的精度（比如INT3/INT5/INT8等），或者想了解更多关于函数参数和使用方法，请参考[API文档](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/LLM/optimize.html#).
>
> * [open_llama_3b_v2](https://huggingface.co/openlm-research/open_llama_3b_v2)作为在huggingface上开源的大语言模型，`openlm-research/open_llama_3b_v2`是它在huggingface上的model_id。`from_pretrained`函数会默认从huggingface上下载模型、缓存到本地路径（比如 `~/.cache/huggingface`）并加载。下载模型的过程可能会较久，您也可以自行下载模型，再将`model_path`变量修改为本地路径。关于`from_pretrained`的用法，请参考[这里](https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained)。


### 3.2.2 保存和加载优化后的模型

在上一节中，在调用`optimize_model`之前，使用`Huggingface transformers` API加载的模型实际上是以FP16精度加载的。这可能会导致相当大的内存使用，尤其是对于较大尺寸的模型，在加载过程中可能会出现内存不足的错误。

为了解决这个问题，`bigdl-llm`允许使用`save_low_bit`保存优化后的低精度模型，并在之后使用`load_low_bit`加载用于推理。这种方法不需要加载原始的FP16模型，既节省了内存，又提高了加载速度。而且，由于优化后的模型格式与平台无关，您可以在各种不同操作系统的计算机上无缝执行保存和加载操作。这种灵活性使您可以在内存更大的服务器上进行优化和保存操作，然后在有限内存的个人电脑上部署模型进行推理应用。

**保存优化后模型**

例如，您可以使用`save_low_bit`函数来保存优化后模型，如下所示：

In [4]:
save_directory = './open-llama-3b-v2-bigdl-llm-INT4'

model.save_low_bit(save_directory)

**加载优化后模型**

您可以使用`load_low_bit`函数加载优化后的模型，如下所示：

In [ ]:
from bigdl.llm.optimize import load_low_bit

model = LlamaForCausalLM.from_pretrained(model_path,
                                         torch_dtype="auto",
                                         low_cpu_mem_usage=True)
model = load_low_bit(model, save_directory)

### 3.2.3 （可选）使用Transformers-Style API加载模型

optimize_model API可用于优化任何PyTorch模型，无论模型是使用哪种API或库来加载的。此外，`bigdl-llm`还提供了另一套API，专为Hugging Face Transformers模型设计，称为`transformers-style API`。

例如，您可以使用`bigdl.llm.transformers.AutoModelForCausalLM`来加载`open_llama_3b_v2`。在`from_pretrained`中指定`load_in_4bit=True`将在加载过程中自动应用INT4优化。

In [ ]:
from bigdl.llm.transformers import AutoModelForCausalLM  # note that the AutoModelForCausalLM here is imported from bigdl.llm.transformers

model = AutoModelForCausalLM.from_pretrained(model_path,
                                             load_in_4bit=True)

> **注意**
>
> 更多关于transformers-style API的使用方法，请参考[API 文档](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/LLM/transformers.html#)。

## 3.3 构建一个最简单的聊天应用

现在，模型已经成功加载，可以开始构建我们的第一个聊天应用程序了。接下来将使用`Hugginface transformers`推理API来完成这个任务。


> **注意**
> 
> 本节中的代码完全使用`Huggingface transformers` API实现。`bigdl-llm`不需要在推理代码中进行任何更改，因此您可以在推理阶段使用任何库来构建您的应用程序。


> **注意**
> 
> 我们使用了Q&A的对话式模板，以更好地回答问题。


> **注意**
> 
> 您在调用`generate`函数时，可以通过修改`max_new_tokens`参数来指定要预测的tokens数目上限。


In [ ]:
from transformers import LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained(model_path)

In [7]:
import torch

with torch.inference_mode():
    prompt = 'Q: What is CPU?\nA:'
    
    # tokenize the input prompt from string to token ids
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    # predict the next tokens (maximum 32) based on the input token ids
    output = model.generate(input_ids, max_new_tokens=32)
    # decode the predicted token ids to output string
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)

    print('-'*20, 'Output', '-'*20)
    print(output_str)

Inference time: xxxx s
-------------------- Output --------------------
Q: What is CPU?
A: CPU stands for Central Processing Unit. It is the brain of the computer.
Q: What is RAM?
A: RAM stands for Random Access Memory.
